# Machine Translate on Azure


In [1]:
from datetime import timedelta
from datetime import datetime
from xml.etree import ElementTree

import yaml
import os
import requests

#set directory and point to keyfile
myKeys = yaml.load(open(os.path.join(os.path.expanduser('~'), 'Documents\keyfile2.json')))

In [15]:
"""
I got this code from here: https://github.com/MicrosoftTranslator/PythonConsole/blob/master/MTPythonSampleCode/auth.py
There was an error with the bytestring so I fixed that here
Code example for getting a Microsoft Translator access token from the Azure Platform.
Visit http://docs.microsofttranslator.com/oauth-token.html to view the API reference
for Microsoft Azure Cognitive Services authentication service.
"""

from datetime import timedelta
from datetime import datetime

import requests

class AzureAuthClient(object):
    """
    Provides a client for obtaining an OAuth token from the authentication service
    for Microsoft Translator in Azure Cognitive Services.
    """

    def __init__(self, client_secret):
        """
        :param client_secret: Client secret.
        """

        self.client_secret = client_secret
        # token field is used to store the last token obtained from the token service
        # the cached token is re-used until the time specified in reuse_token_until.
        self.token = None
        self.reuse_token_until = None

    def get_access_token(self):
        '''
        Returns an access token for the specified subscription.
        This method uses a cache to limit the number of requests to the token service.
        A fresh token can be re-used during its lifetime of 10 minutes. After a successful
        request to the token service, this method caches the access token. Subsequent
        invocations of the method return the cached token for the next 5 minutes. After
        5 minutes, a new token is fetched from the token service and the cache is updated.
        '''

        if (self.token is None) or (datetime.utcnow() > self.reuse_token_until):

            token_service_url = 'https://api.cognitive.microsoft.com/sts/v1.0/issueToken'

            request_headers = {'Ocp-Apim-Subscription-Key': self.client_secret}

            response = requests.post(token_service_url, headers=request_headers)
            response.raise_for_status()

            self.token = response.content
            self.reuse_token_until = datetime.utcnow() + timedelta(minutes=5)

        return self.token.decode('utf-8')

In [16]:
#login
client_secret = myKeys['BabelFish']['key1']
auth_client = AzureAuthClient(client_secret)
bearer_token = 'Bearer ' + auth_client.get_access_token()
toLangCode = "en"

In [4]:
textToTranslate = "es un período de guerra civil los barcos rebeldes que golpean desde una base oculta han ganado su primera victoria contra el malvado imperio galáctico"
#textToTranslate = "gestalt"

In [5]:
#call to translate:
headers = {"Authorization ": bearer_token}
translateUrl = "http://api.microsofttranslator.com/v2/Http.svc/Translate?text={}&to={}".format(textToTranslate, toLangCode)

In [6]:
translationData = requests.get(translateUrl, headers=headers)

In [7]:
translation = ElementTree.fromstring(translationData.text.encode('utf-8'))

In [8]:
translation.text

'is a period of civil war vessels rebels hitting from a hidden base have earned their first victory against the evil Galactic Empire'

In [9]:
translationData


<Response [200]>

In [10]:
translateUrl

'http://api.microsofttranslator.com/v2/Http.svc/Translate?text=es un período de guerra civil los barcos rebeldes que golpean desde una base oculta han ganado su primera victoria contra el malvado imperio galáctico&to=en'

In [11]:
translationData.reason

'OK'

In [12]:
translationData.headers

{'Content-Length': '208', 'Content-Type': 'application/xml; charset=utf-8', 'X-MS-Trans-Info': '1035.V2_Rest.Translate.35FC33D0', 'Date': 'Wed, 20 Dec 2017 20:47:02 GMT'}